In [2]:
%pip install pyarrow
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
  Using cached pyarrow-17.0.0.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic/numpy-1.26.4+computecanada-cp311-cp311-linux_x86_64.whl (from pyarrow)
  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [402 lines of output]
      <string>:34: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      running bdist_wheel
      running build
      runnin

In [4]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

ModuleNotFoundError: No module named 'datasets'

Load the dataset

In [ ]:
# Load the dataset
dataset = load_dataset("khairi/uniprot-swissprot")

Model

In [ ]:
# Define the model checkpoint
model_checkpoint = "dmis-lab/biobert-base-cased-v1.2"

# Define label maps
labels = dataset['train'].features['label'].names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# Generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=len(labels), id2label=id2label, label2id=label2id
)

In [ ]:
# Display architecture
model

Preprocess data

In [ ]:
# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained("dotan1111/BioTokenizer-BFD-BPE-100")

In [ ]:
# Create tokenize function
def tokenize_function(examples):
    # Extract text
    text = examples["sequence"]
    
    # Tokenize and truncate text
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )
    
    return tokenized_inputs

In [ ]:
# Tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)

In [ ]:
# Create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Evaluation

In [ ]:
# Import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [ ]:
# Define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

Apply untrained model to text

In [ ]:
# Define list of examples
text_list = ["MVLSPADKTNVKAAW", "GAGGAGAAGGTGTGGCG", "MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNEKAGA"]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # Tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # Compute logits
    logits = model(inputs).logits
    # Convert logits to label
    predictions = torch.argmax(logits)
    
    print(text + " - " + id2label[predictions.tolist()])

Train model

In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                         r=4,
                         lora_alpha=32,
                         lora_dropout=0.01,
                         target_modules=['q_lin'])

In [ ]:
peft_config

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# Hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# Create trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

Generate prediction

In [ ]:
model.to('cpu') # moving to cpu

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")
    
    logits = model(inputs).logits
    predictions = torch.argmax(logits, dim=1)
    
    print(text + " - " + id2label[predictions.tolist()[0]])